<a href="https://colab.research.google.com/github/taimoorsardar/Deep-Learning-Projects/blob/main/TextSummarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# lets install some of the packages
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 58.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 48.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 16.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 73.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 55.8 MB/s eta 0:00:0

checking the gpu

In [ ]:
# to check which gpu we got
!nvidia-smi # not important

Wed Aug  9 10:35:29 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

importing essential packages and libraries

In [ ]:
from transformers import pipeline, AutoModelForSeq2SeqLM, AutoTokenizer

import matplotlib.pyplot as plt
from datasets import load_dataset, load_metric
import pandas as pd
import numpy as np

# nltk is used for data analysis, data cleaning,tokenizing etc in nlp
import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import torch


# Punkt is made to learn parameters from a corpus in an unsupervised way that is related to the target domain, such as a list of abbreviations, acronyms, etc.
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

setting up the device

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

setting up the model and tokenizer

In [ ]:
model_ckpt = "google/pegasus-cnn_dailymail"

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model

PegasusForConditionalGeneration(
  (model): PegasusModel(
    (shared): Embedding(96103, 1024, padding_idx=0)
    (encoder): PegasusEncoder(
      (embed_tokens): Embedding(96103, 1024, padding_idx=0)
      (embed_positions): PegasusSinusoidalPositionalEmbedding(1024, 1024)
      (layers): ModuleList(
        (0-15): 16 x PegasusEncoderLayer(
          (self_attn): PegasusAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_no

loading the dataset

In [ ]:
dataset_cnn = load_dataset("cnn_dailymail", '3.0.0')
dataset_cnn

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [ ]:
split_lengths = [len(dataset_cnn[split]) for split in dataset_cnn]
split_lengths

[287113, 13368, 11490]

In [ ]:
print(f"Features : {dataset_cnn['train'].column_names}")

Features : ['article', 'highlights', 'id']


In [ ]:
# lets check 1 example
print("\nArticle")
print(dataset_cnn['test']['article'][1])

print("\nHighlights")
print(dataset_cnn['test']['highlights'][1])


Article
(CNN)Never mind cats having nine lives. A stray pooch in Washington State has used up at least three of her own after being hit by a car, apparently whacked on the head with a hammer in a misguided mercy killing and then buried in a field -- only to survive. That's according to Washington State University, where the dog -- a friendly white-and-black bully breed mix now named Theia -- has been receiving care at the Veterinary Teaching Hospital. Four days after her apparent death, the dog managed to stagger to a nearby farm, dirt-covered and emaciated, where she was found by a worker who took her to a vet for help. She was taken in by Moses Lake, Washington, resident Sara Mellado. "Considering everything that she's been through, she's incredibly gentle and loving," Mellado said, according to WSU News. "She's a true miracle dog and she deserves a good life." Theia is only one year old but the dog's brush with death did not leave her unscathed. She suffered a dislocated jaw, leg i

In [ ]:
######################################################################
#####        WHILE DOING ANY TASK ALWAYS CHECK FIRST WHETHER YOUR PRETRAINED MODEL CAN WORK ON THIS DATASET OR NOT,
#####        IF IT IS DOING GOOD NO NEED TO TRAIN, ELSE TRAIN,
#####        THIS WAS TAKING TO MUCH MEMORY SO I COMMENTED IT OUT
######################################################################

In [ ]:
#article = dataset_cnn['test']['article'][1]
#highlight = dataset_cnn['test']['highlights'][1]

In [ ]:
#pipe = pipeline('summarization', model = model_ckpt)

In [ ]:
#pipe_out = pipe(article)
#pipe_out
'''not so good performance'''

'not so good performance'

lets check accuracy of pre-trained model (before fine tuning)

In [ ]:
'''
these functions are nearly same in every summarization tasks
more like use it as a template
'''

# this function will generate batch
def generate_batch_sized_chunks(list_of_elements, batch_size):
  '''split the dataset into smaller batches that we can  process simultaneously
  yield successive batch-sized chunks from list of elements.'''
  for i in range(0, len(list_of_elements), batch_size):
    yield list_of_elements[i : i + batch_size]


def calculate_metric_on_test_ds(dataset, metric, model, tokenizer, batch_size = 16, device = device, column_text = 'article', column_summary = 'highlights'):
  article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
  target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

  for article_batch, target_batch in tqdm(zip(article_batches, target_batches), total = len(article_batches)):

    inputs = tokenizer(article_batch, max_length = 1024, truncation = True, padding = 'max_length', return_tensors = 'pt')
    summaries = model.generate(input_ids = inputs['input_ids'].to(device),
                               attention_mask = inputs['attention_mask'].to(device),
                               length_penalty = 0.8, num_beams = 8, max_length = 128)
    '''parameters for length penalty ensures that the model does not generate sequences that are too long'''

    # finally, we decode the generated texts,
    # replace the token, and add the decoded texts with the references to the metric
    decoded_summaries = [tokenizer.decode(s, skip_special_tokens = True, clean_up_tokenization_spaces = True) for s in summaries]

    decoded_summaries = [d.replace("", " ") for d in decoded_summaries]

    metric.add_batch(predictions = decoded_summaries, references = target_batch)

  # finally compute and return the ROUGE scores
  score = metric.compute()
  return score

In [ ]:
# for summarization tasks, we have rouge metric


# because of excessive gpu usage  am not running this cell


'''rouge_metric = load_metric('rouge')

score = calculate_metric_on_test_ds(dataset_cnn['test'], rouge_metric, model, tokenizer, column_text = 'article', column_summary = 'highlights', batch_size = 8)
'''

"rouge_metric = load_metric('rouge')\n\nscore = calculate_metric_on_test_ds(dataset_cnn['test'], rouge_metric, model, tokenizer, column_text = 'article', column_summary = 'highlights', batch_size = 8)\n"

In [ ]:
# lets print out some of the metrics now
'''
rouge_names = ['rouge1', 'rouge2','rougeL', 'rougeLsum']
rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)

pd.DataFrame(rouge_dict, index = ['pegasus'])
'''
''' all of this till now is the evaluation technique of summarization model'''

' all of this till now is the evaluation technique of summarization model'

now lets work on fine tuning

we will start with tokenizng the training dataset

In [ ]:
'''this function converts the training dataset to numerical presentation (tokens)'''
def convert_examples_to_features(example_batch):

  input_encodings = tokenizer(example_batch['article'], max_length = 1024, truncation = True)

  with tokenizer.as_target_tokenizer():
    target_encodings = tokenizer(example_batch['highlights'], max_length = 128, truncation = True)

  return {
      'input_ids' : input_encodings['input_ids'],
      'attention_mask' : input_encodings['attention_mask'],
      'labels' : target_encodings['input_ids']
  }

dataset_cnn_pt = dataset_cnn.map(convert_examples_to_features, batched = True)

Map:   0%|          | 0/287113 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3635: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/13368 [00:00<?, ? examples/s]

Map:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [ ]:
# now lets check the tokens
dataset_cnn_pt['train'][0]

In [ ]:
from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model = model)

In [ ]:
'''
the combination of TrainingArguments and Trainer classes from the Hugging Face transformers library allows you to
easily configure training hyperparameters and manage the training loop for transformer-based models like Pegasus,
which is particularly useful for NLP tasks like text summarization.
'''
from transformers import TrainingArguments, Trainer

'''allows you to define various training configurations and hyperparameters for training your model'''
trainer_args = TrainingArguments(
    output_dir = 'pegasus-cnn', num_train_epochs = 1, warmup_steps = 500,
    per_device_train_batch_size = 1, per_device_eval_batch_size = 1,
    weight_decay = 0.01, logging_steps = 10, evaluation_strategy = 'steps',
    eval_steps = 500, save_steps = 1e6, gradient_accumulation_steps = 16)



In [ ]:
'''
Trainer is another class provided by the transformers library that simplifies the training loop for you.
It abstracts away many of the details of the training process, like forwarding inputs through the model, calculating losses, and updating gradients.
You provide it with your model, training data, and the TrainingArguments object, and it handles the training process for you.
'''
trainer = Trainer(model = model, args = trainer_Args,
                  tokenizer = tokenizer, data_collator = seq2seq_data_collator,
                  train_dataset = dataset_cnn_pt['train'],
                  eval_dataset = dataset_cnn_pt['validation'])

In [ ]:
# this line will now train the model
trainer.train()

In [ ]:
# once trained, again check accuracy
score = calculate_metric_on_test_ds(
    dataset_cnn['test'], rouge_metric, trainer.model, tokenizer, batch_size = 2, column_text = 'article', column_summary = 'highlights')

rouge_dict = dict((rn, score[rn].mid.features) for rn in rouge_names)
pd.DataFrame(rouge_dict, index = [f'pegasus'])

In [ ]:
# save model
 torch.save(trainer.model.state_dict(), 'pegasus_cnn.model')

# INFERENCE

In [ ]:
# load model
# model.load_state_dict(torch.load('/content/pegasus_cnn.model'))